In [1]:
import pandas as pd
import warnings
import csv
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
warnings.filterwarnings("ignore") 

# 1. Data Preprocesing

In [2]:
data = [['Milk','Onion','Nutmeg','Kidney Beans','Eggs','Yougurt'],
      ['Dill','Onion','Nutmeg','Kidney Beans','Eggs','Yougurt'],
      ['Milk','Apple','Kidney Beans','Eggs'],
      ['Milk','Unicorn','Corn','Kidney Beans','Yougurt'],
      ['Corn','Onion','Kidney Beans','Ice Cream','Eggs']]

data

[['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yougurt'],
 ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yougurt'],
 ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
 ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yougurt'],
 ['Corn', 'Onion', 'Kidney Beans', 'Ice Cream', 'Eggs']]

In [3]:
# Data perlu diubah ke dalam bentuk dataframe
# kolom menunjukkan list unique item, sedangkan baris menunjukkan ada/tidaknya item tersebut

trf = TransactionEncoder()
data_trf = trf.fit(data).transform(data)

df = pd.DataFrame(data_trf,columns=trf.columns_)
df

,Apple,Corn,Dill,Eggs,Ice Cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yougurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


# 2. Modeling

## 2.1 Menghitung nilai support

In [4]:
# Nilai Support Kidney Beans = 1 artinya item tersebut selalu muncul di setiap transaksi
# Egg hampir selalu muncul di setiap transaksi
frequent_itemsets = apriori(df, min_support=0.047, use_colnames=True)
frequent_itemsets.sort_values('support',ascending=False,inplace=True)
frequent_itemsets

,support,itemsets
5,1.0,(Kidney Beans)
3,0.8,(Eggs)
27,0.8,"(Kidney Beans, Eggs)"
30,0.6,"(Eggs, Onion)"
8,0.6,(Onion)
...,...,...
56,0.2,"(Kidney Beans, Onion, Corn)"
55,0.2,"(Milk, Kidney Beans, Corn)"
54,0.2,"(Ice Cream, Onion, Corn)"
53,0.2,"(Ice Cream, Kidney Beans, Corn)"


## 2.2 Pembentukan aturan
- Jika rules yang terbentu adalah A -> B, maka A disebut antecedents, sedangkan B disebut consequents
- Tidak hanya itemlist, setiap rules juga akan memiliki nilai support dan confidence
- lift dapat menunjukkan kekuatan aturan yang dibuat. semakin besar nilai lift, semakin bagus aturan tersebut
- leverage hampir sama dengan lift, namun yang dihitung adalah perbedaannya.
- nilai conviction yang tinggi berarti consequents sangat tergantung pada antecedents.

In [5]:
# Interpretasi Rules
# 1.Orang yang membeli (Kidney Beans) akan membeli (Eggs)
# 2.Orang yang membeli (Eggs) akan membeli (Kidney Beans)
rules1 = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules1.sort_values('lift',ascending=False,inplace=True)
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
263,"(Kidney Beans, Onion, Corn)",(Ice Cream),0.2,0.2,0.2,1.0,5.0,0.16,inf
706,"(Milk, Yougurt, Corn)","(Kidney Beans, Unicorn)",0.2,0.2,0.2,1.0,5.0,0.16,inf
701,"(Milk, Kidney Beans, Unicorn)","(Yougurt, Corn)",0.2,0.2,0.2,1.0,5.0,0.16,inf
679,"(Kidney Beans, Onion, Corn)","(Ice Cream, Eggs)",0.2,0.2,0.2,1.0,5.0,0.16,inf
308,(Ice Cream),"(Kidney Beans, Eggs, Corn)",0.2,0.2,0.2,1.0,5.0,0.16,inf
...,...,...,...,...,...,...,...,...,...
1068,(Kidney Beans),"(Ice Cream, Onion)",1.0,0.2,0.2,0.2,1.0,0.00,1.0
381,"(Milk, Yougurt, Eggs, Nutmeg)",(Kidney Beans),0.2,1.0,0.2,1.0,1.0,0.00,inf
379,(Kidney Beans),"(Milk, Nutmeg, Eggs, Onion)",1.0,0.2,0.2,0.2,1.0,0.00,1.0
919,(Kidney Beans),"(Nutmeg, Dill, Yougurt)",1.0,0.2,0.2,0.2,1.0,0.00,1.0


In [7]:
# Misal kita ingin membuat aturan lainnya yaitu
# 1. minimum antecedentnya adalah tiga
# 2. confidence > 0.8
# 3. lift >  2.5

rules1['total_antecedents'] = rules1['antecedents'].apply(lambda x: len(x))
rules2 = rules1[(rules1.total_antecedents >= 3) & 
                (rules1.confidence > 0.8) & 
                (rules1.lift > 2.5)]

rules2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,total_antecedents
263,"(Kidney Beans, Onion, Corn)",(Ice Cream),0.2,0.2,0.2,1.0,5.0,0.16,inf,3
706,"(Milk, Yougurt, Corn)","(Kidney Beans, Unicorn)",0.2,0.2,0.2,1.0,5.0,0.16,inf,3
701,"(Milk, Kidney Beans, Unicorn)","(Yougurt, Corn)",0.2,0.2,0.2,1.0,5.0,0.16,inf,3
679,"(Kidney Beans, Onion, Corn)","(Ice Cream, Eggs)",0.2,0.2,0.2,1.0,5.0,0.16,inf,3
677,"(Ice Cream, Kidney Beans, Onion)","(Eggs, Corn)",0.2,0.2,0.2,1.0,5.0,0.16,inf,3
676,"(Kidney Beans, Eggs, Corn)","(Ice Cream, Onion)",0.2,0.2,0.2,1.0,5.0,0.16,inf,3
217,"(Milk, Kidney Beans, Corn)",(Unicorn),0.2,0.2,0.2,1.0,5.0,0.16,inf,3
674,"(Ice Cream, Kidney Beans, Eggs)","(Onion, Corn)",0.2,0.2,0.2,1.0,5.0,0.16,inf,3
301,"(Kidney Beans, Eggs, Corn)",(Ice Cream),0.2,0.2,0.2,1.0,5.0,0.16,inf,3
668,"(Kidney Beans, Eggs, Onion, Corn)",(Ice Cream),0.2,0.2,0.2,1.0,5.0,0.16,inf,4
